# 抓三個種類新聞

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent


In [4]:
news_links =['game-news', 'game-tips', 'esports']
news_categories=['遊戲新聞','遊戲攻略','電競賽事']
base_url = 'https://tw.news.yahoo.com/'


In [5]:
user_agent = UserAgent()
user_agent.random

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'

# 存放資料之變數

In [6]:
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []

In [7]:
for i, url_short_name in enumerate(news_links):  #針對每一類 共有3類

    category = news_categories[i]  #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name 
    print("Getting categorical news:", category)
    print(category_url)
    # Request the categorical news page
    # req = requests.get(category_url)
    req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    page = BeautifulSoup(req.text, 'html.parser')

    # 抓新聞列表
    items = page.find_all('li', class_='stream-item')
    #print(items)


    # Let's start to crawl the news in the first page for that category
    serial_no = 1
    for item_j, item in enumerate(items,start=1): #針對每一篇項目 抓其細節
        title = item.find('h3').text
        print(serial_no,'--', title )
        
        link = item.find('a').get('href')
        link = "https://tw.news.yahoo.com"+link
        print(link)

        # 儲存圖片網址變數
        image_url = None

# **方法 1: 嘗試直接抓取 <figure> 內的 <img>**
        figure_tag = page.find("figure", class_="caas-figure")
        if figure_tag:
            img_tag = figure_tag.find("img")
            if img_tag:
                image_url = img_tag.get("src") or img_tag.get("data-src")  # 優先抓 src，若無則抓 data-src


# **Yahoo 可能返回錯誤的圖片網址 (包含兩個 URL)，需要修正**
        if image_url and "https://s.yimg.com/ny/api/res/" in image_url:
            match = re.search(r'(https://s\.yimg\.com/os/creatr-uploaded-images/[\w./-]+)', image_url)
            if match:
                image_url = match.group(0)

        
       
      
        #print(f'發布於:'+news_date)
        categories.append(category)
        titles.append(title)
        links.append(link)
        #photo_links.append(image_url)
        
        
        req = requests.get(link, headers={ 'user-agent': user_agent.random }, timeout=5)
        
        page = BeautifulSoup(req.text,'html.parser')
        time_element = page.find('time', {'class': "caas-attr-meta-time"})

        if time_element:
            news_time_iso = time_element['datetime']  # 擷取 ISO 8601 格式，如："2025-02-26T07:12:09.000Z"

            news_time = datetime.strptime(news_time_iso, "%Y-%m-%dT%H:%M:%S.%fZ")

    # 轉換格式
            dtime = news_time.strftime("%Y/%m/%d %H:%M")  # 轉換為 "YYYY/MM/DD HH:MM" 格式
            news_date = news_time.strftime("%Y-%m-%d")  # 轉換為 "YYYY-MM-DD" 格式
        else:
            dtime = "未知"
            news_date = "未知"
        print(f"發布日期"+news_date)
        dates.append(news_date)
        
        # 儲存圖片網址變數
        image_url = None

# **方法 1: 嘗試直接抓取 <figure> 內的 <img>**
        figure_tag = page.find("figure", class_="caas-figure")
        if figure_tag:
            img_tag = figure_tag.find("img")
            if img_tag:
                image_url = img_tag.get("src") or img_tag.get("data-src")  # 優先抓 src，若無則抓 data-src


# **Yahoo 可能返回錯誤的圖片網址 (包含兩個 URL)，需要修正**
        if image_url and "https://s.yimg.com/ny/api/res/" in image_url:
            match = re.search(r'(https://s\.yimg\.com/os/creatr-uploaded-images/[\w./-]+)', image_url)
            if match:
                image_url = match.group(0)
        photo_links.append(image_url)
                
        item_id.append(url_short_name + "_" + news_date + "_" + str(serial_no))
        serial_no += 1
        # Find content
        content = [p.text.strip() for p in page.select(".caas-body p")]
        filtered_news = [item for item in content if not item.startswith("相關新聞")]
       
        contents.append(filtered_news)
        
        if item_j >= 4: # Here we crawl only 4 pieces of news for each category, in order to save time.
            break 
        

Getting categorical news: 遊戲新聞
https://tw.news.yahoo.com/game-news
1 -- 《戰神》正式邁入弒神之旅20週年，官方免費推遊戲內套裝、桌布供粉絲下載，還有紀念T等週邊讓玩家入手
https://tw.news.yahoo.com/%E3%80%8A%E6%88%B0%E7%A5%9E%E3%80%8B%E6%AD%A3%E5%BC%8F%E9%82%81%E5%85%A5%E5%BC%92%E7%A5%9E%E4%B9%8B%E6%97%8520%E9%80%B1%E5%B9%B4%EF%BC%8C%E5%AE%98%E6%96%B9%E5%85%8D%E8%B2%BB%E6%8E%A8%E9%81%8A%E6%88%B2%E5%85%A7%E5%A5%97%E8%A3%9D%E3%80%81%E6%A1%8C%E5%B8%83%E4%BE%9B%E7%B2%89%E7%B5%B2%E4%B8%8B%E8%BC%89%EF%BC%8C%E9%82%84%E6%9C%89%E7%B4%80%E5%BF%B5t%E7%AD%89%E9%80%B1%E9%82%8A%E8%AE%93%E7%8E%A9%E5%AE%B6%E5%85%A5%E6%89%8B-031018902.html
發布日期2025-03-22
2 -- 《鳴潮》宣布4月29日登上Steam！2.2版前瞻兌換碼公開
https://tw.news.yahoo.com/%E3%80%8A%E9%B3%B4%E6%BD%AE%E3%80%8B%E5%AE%A3%E5%B8%834%E6%9C%8829%E6%97%A5%E7%99%BB%E4%B8%8Asteam%EF%BC%8122%E7%89%88%E5%89%8D%E7%9E%BB%E5%85%8C%E6%8F%9B%E7%A2%BC%E5%85%AC%E9%96%8B-192145445.html
發布日期2025-03-21
3 -- 《Pokemon TCG Pocket》新卡包「嗨放異彩」3月27日上線！收錄作品曝光，異色寶可夢首次登場
https://tw.news.yahoo.com/%E3%80%8Apokemon-tcg-pocket%E3%80%8B%E6%96%B0

# Save data

In [8]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])
df.head(2)
df.shape
df.content[0]
df.to_csv("cna_category_news.csv", sep="|", index=False)

# tokenize news and save

In [9]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('cna_category_news.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(w, p)) for w, p in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc', 'VC']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('cna_news_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

d:\project_code\bigdata_class_v1\venv_bigdata\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Inference: 100%|██████████| 1/1 [00:00<00:00,  4.66it/s]

Tokenize OK!
CPU times: total: 9.62 s
Wall time: 13.8 s


# Count top keywords

In [10]:
from collections import Counter
import pandas as pd

# 
df = pd.read_csv('cna_news_preprocessed.csv',sep='|')
news_categories=['遊戲新聞','遊戲攻略','電競賽事']
# Filter condition: two words and specified POS
# 過濾條件:兩個字以上 特定的詞性
allowedPOS=['Na','Nb','Nc']

# 
# get topk keyword function
def get_top_words():
    top_cate_words={} # final result
    counter_all = Counter() # counter for category '全部'
    for category in news_categories:

        df_group = df[df.category == category]

        # concatenate all filtered words in the same category
        words_group = []
        for row in df_group.token_pos:

            # filter words for each news
            filtered_words =[]
            for (word, pos) in eval(row):
                if (len(word) >= 2) & (pos in allowedPOS):
                    filtered_words.append(word)

            # concatenate filtered words  
            words_group += filtered_words

        # now we can count word frequency
        counter = Counter( words_group )

        # counter 
        counter_all += counter
        topwords = counter.most_common(100)

        # store topwords
        top_cate_words[category]= topwords

    # Process category '全部'
    top_cate_words['全部'] = counter_all.most_common(100)
    
    # To conveniently save data using pandas, we should convert dict to list.
    return list(top_cate_words.items())

# Save top 200 word frequency for each category
top_group_words = get_top_words()
df_top_group_words = pd.DataFrame(top_group_words, columns = ['category','top_keys'])
df_top_group_words.to_csv('cna_news_topkey_with_category_via_token_pos.csv', index=False)